In [1]:
import sys
sys.path.append('../../')

In [2]:
# imports
import os
import sys
import math
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from distnet import DistResNeXt50
from dataset import KITTI,NuScenes
import matplotlib.patches as patches
import matplotlib.patheffects as patheffects
import pandas as pd
import metrics

In [3]:
class FeatureExtractor(DistResNeXt50):
    def __init__(self, n_classes, image_size, pretrained, keypoints):
        super(FeatureExtractor, self).__init__(n_classes, image_size, pretrained, keypoints)
    
    def forward(self, x, x_boxes):
        
        # feed entire image thru the backbone
        x = self.backbone(x)
        
        # perform roi pooling of resnext features
        x = self.RoIPool(x, x_boxes)
        
        # flatten our feature vector
        x = x.flatten(start_dim=1)
        
        return x

In [4]:
# define the device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
split = 'train'
# split = 'test'

In [6]:
# create our transforms routine
size=1024
transforms = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                             torchvision.transforms.Normalize(
                                                 mean=[0.485, 0.456, 0.406],
                                                 std=[0.229, 0.224, 0.225]
                                             )
                                            ])

# create test dataset and loader
dataset = NuScenes(img_dir='/irad_mounts/lambda-quad-5-data/beholder/intern_data/nuscenes-full/',
                          meta_path='/irad_mounts/lambda-quad-5-data/beholder/intern_data/nuscenes-full/nuscenes-v1.0.csv',
                          split = split,
                          transforms = transforms,
                          size = size)

# 17 classes of objects in nuscenes full dataset
n_classes = dataset.n_classes

706639


In [7]:
model = FeatureExtractor(n_classes, size, False, True)
model.to(device)
# model.load_state_dict(torch.load('/irad_users/determined/checkpoints/82cf578e-200c-4909-b7a4-2324103a0ac1/state_dict.pth')['models_state_dict'][0])
# model.load_state_dict(torch.load('/irad_users/determined/checkpoints/52b28b81-b8ae-4af6-ba8d-14220cd649d5/state_dict.pth')['models_state_dict'][0])
# model.load_state_dict(torch.load('/irad_users/determined/checkpoints/d5c59a17-dbd6-4809-8d9a-1dc7f4a8e560/state_dict.pth')['models_state_dict'][0])
model.load_state_dict(torch.load('/irad_users/determined/checkpoints/1f6a5c6d-bfb9-497e-b27e-baed4b642d39/state_dict.pth')['models_state_dict'][0])

<All keys matched successfully>

In [ ]:
# reconstruct dataframe w/ feature vectors added
df = []
model.eval()
with torch.no_grad():
    for i in range(len(dataset)):
        print(f'\r{i}/{len(dataset)-1}', end='')

        # grab the batch and move the batch to the gpu
        data = dataset.__getitem__(i)
        inputs, boxes, distances, classes = data
        inputs = inputs.to(device).unsqueeze(0)
        boxes = [boxes.to(device)]

        # forward pass
        fv = model(inputs, boxes).cpu().numpy()
        
        classes = classes.cpu().numpy()
        
        # construct a dictionary for each annotation item
        # append it to larger dataframe
        distances = distances.cpu().numpy()
        for i in range(boxes[0].shape[0]):
            bboxes = boxes[0][i].cpu().numpy()
            d = {
                'boxes_xmin': bboxes[0],
                'boxes_ymin': bboxes[1],
                'boxes_xmax': bboxes[2],
                'boxes_ymax': bboxes[3],
                'classes': classes[i],
                'feature_vector': fv[i],
                'distances': distances[i]
            }
            df.append(d)

1/128112

../../dataset.py:109: PerformanceWarning: indexing past lexsort depth may impact performance.
  data = self.metadata.loc[self.sample_list[idx]]
/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


47654/128112

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
# output data to csv
df = pd.DataFrame(df)
df.to_json(f'{split}.json')